# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio


from libs.messaging_service import MessagingService

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://demo-participant-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Instatiate Messaging Service

In [4]:
messaging_service = MessagingService(agent_controller)

## Optional - Configure a Default Mediator (You only need to do this Once)

We have hosted an ACA agent in open mediation mode. Accept this connection and request mediation from it and it should grant your request. If you set this as your default mediation all future invites will have serviceEndpoints pointing to this agent which will then forward messages on to your endpoint.

In [16]:
## If this returns a mediator then you do not need to run this again.
await agent_controller.mediation.get_default_mediator()

{'mediator_terms': [],
 'endpoint': 'http://139.162.224.50:6666',
 'recipient_terms': [],
 'state': 'granted',
 'updated_at': '2021-06-08 07:35:54.139601Z',
 'created_at': '2021-06-08 07:35:53.645188Z',
 'mediation_id': 'b5ffd5f3-f846-43a9-a7dd-98b610fbccf8',
 'role': 'client',
 'connection_id': '930dd81d-afc4-45d0-a6da-e231ac3d9b78',
 'routing_keys': ['BQKTbHwJap7Su9ZtkBJgqnHEAQt4ysjygBkAEjEngMfp']}

In [9]:
mediator_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bb47c3ea-ab9a-4ef4-984d-409cb4909956', 'recipientKeys': ['HkwRkbCGEXn4LviXNen5f8kqQ1WxVRpQQGxaQG7PuSZD'], 'serviceEndpoint': 'http://139.162.224.50:6666', 'label': 'HGF_ACA_Playground_Mediator'}

In [10]:
invitation_response = await agent_controller.connections.receive_invitation(mediator_invite)

In [13]:
connection_id = invitation_response["connection_id"]
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    

In [14]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_record_by_id(mediation_req["mediation_id"])
print(f"State : {records_response['state']}")

State : granted


In [15]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(records_response['mediation_id'])
print(default_mediation_res)

{'mediator_terms': [], 'endpoint': 'http://139.162.224.50:6666', 'recipient_terms': [], 'state': 'granted', 'updated_at': '2021-06-08 07:35:54.139601Z', 'created_at': '2021-06-08 07:35:53.645188Z', 'mediation_id': 'b5ffd5f3-f846-43a9-a7dd-98b610fbccf8', 'role': 'client', 'connection_id': '930dd81d-afc4-45d0-a6da-e231ac3d9b78', 'routing_keys': ['BQKTbHwJap7Su9ZtkBJgqnHEAQt4ysjygBkAEjEngMfp']}


## Load Contacts from File

You will need to have saved them to file first. This is how you can persist state.

If you want to message these contacts you just loaded you will need to access their connection id's. Probably the easiest way is to just copy them.



In [ ]:
messaging_service.load_from_file()

## Add Contact

### Either from Invitation Shared

In [32]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "77eb2de5-f588-4c56-a3a8-40d6dd36fcdb", "label": "Hyperledger Global Forum", "serviceEndpoint": "https://073df57eb62d.ngrok.io", "recipientKeys": ["5xzLTBet1fXQR8Vmdyt4VtiSE7CxVLpjWsXVJJXa7HUY"]}


In [33]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "HLGF")

Contact with ID dbc1b539-5d6e-4a73-9eb8-e758f2b7d6ff and alias HLGF added
Connection with ID: dbc1b539-5d6e-4a73-9eb8-e758f2b7d6ff is now active.
Contact HLGF Added


### Or by Creating and Sharing an Invite

In [17]:
alias = "HLGF"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

Contact with ID 638257e4-3241-4d72-a51c-c376a0f87a5b and alias HLGF added
----------------------------------------------------------------------------------------------------

Share this invite object with another entity you wish to add as a contact under alias HLGF

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "79690dce-73e8-40fe-943d-212771bfce94", "recipientKeys": ["GjM12SGxGiyN4TJ3XLda8AjNUPSxsS8byDnfN4Uvpv5H"], "serviceEndpoint": "http://139.162.224.50:6666", "routingKeys": ["BQKTbHwJap7Su9ZtkBJgqnHEAQt4ysjygBkAEjEngMfp"], "label": "<Label YOUR Agent>"}


----------------------------------------------------------------------------------------------------
Connection with ID: 638257e4-3241-4d72-a51c-c376a0f87a5b is now active.
Contact HLGF Added
Message received from connection 638257e4-3241-4d72-a51c-c376a0f87a5b with alias HLGF


## Send Message to Contact by Connection ID

In [35]:
some_arbitrary_sized_message = "Hey how are you, want to grab a coffee?"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

Message received from connection dbc1b539-5d6e-4a73-9eb8-e758f2b7d6ff with alias HLGF
Message received from connection dbc1b539-5d6e-4a73-9eb8-e758f2b7d6ff with alias HLGF


## Display Inbox for Contact

In [36]:
messaging_service.display_inbox_for_contact(new_contact_id)

----------------------------------------------------------------------------------------------------
Inbox for Contact HLGF
----------------------------------------------------------------------------------------------------
Connection ID : dbc1b539-5d6e-4a73-9eb8-e758f2b7d6ff 
Agent Label : Hyperledger Global Forum 
Total Messages : 4
----------------------------------------------------------------------------------------------------
Received                                                                              Sent
----------------------------------------------------------------------------------------------------
Tue Jun  8 07:55:46 2021
                                                            Hey how are you, want to grab a coffee? 
--------------------------------------------------
Tue Jun  8 07:55:49 2021
                                                            Hey how are you, want to grab a coffee? 
--------------------------------------------------
Tue Jun  8 07:5

## Get Contacts

In [ ]:
messaging_service.view_inbox(show_inactive=True)

In [ ]:
response = messaging_service.get_contacts()
for contact in response:
    print(f"{contact.alias} : {contact.connection_id}")
    print(contact.is_active.done())
    print(contact.__dict__)

## Save Contacts

If you restart this notebook kernal, or need to stop and restart the docker images all contacts and inbox information will be lost. Use the save_to_file function to persist this data in a file.

In [ ]:
messaging_service.save_to_file()

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()